In [1]:
import pandas as pd
import sqlite3
import re
import unidecode

##### Scrape and load the Wiki data into a Pandas dataframe

In [2]:
capitals_url = 'https://en.wikipedia.org/wiki/List_of_national_capitals'
list_of_tables = pd.read_html(capitals_url)
capitals_df = list_of_tables[1]

cities_url = 'https://en.wikipedia.org/wiki/List_of_cities_by_elevation'
list_of_tables = pd.read_html(cities_url)
cities_df = list_of_tables[1]

In [3]:
capitals_df

,City/Town,Country/Territory,Notes
0,Abidjan (former capital; still hosts some gove...,Ivory Coast (Côte d'Ivoire),Abidjan is the largest city in Ivory Coast and...
1,Yamoussoukro (de jure),Ivory Coast (Côte d'Ivoire),Abidjan is the largest city in Ivory Coast and...
2,Abu Dhabi,United Arab Emirates,NaN
3,Abuja,Nigeria,Lagos was the capital from 1914 to 1991.
4,Accra,Ghana,NaN
...,...,...,...
255,Windhoek,Namibia,NaN
256,Yaoundé,Cameroon,NaN
257,Yaren (de facto),Nauru,"Nauru has no official capital; however, the go..."
258,Yerevan,Armenia,NaN


##### Clean the column names

In [4]:
capitals_df.rename(columns={'City/Town':'city'},inplace=True)
capitals_df.rename(columns={'Country/Territory':'country'},inplace=True)
capitals_df.rename(columns={'Notes':'notes'},inplace=True)

# extract all the data before the extra information
capitals_df['city'] = capitals_df['city'].apply(lambda x: re.search('^(.+?)\(',str(x)).group(1).strip() if str(x).find('(') > 0 else x)
capitals_df['country'] = capitals_df['country'].apply(lambda x: re.search('^(.+?)\[',str(x)).group(1).strip() if str(x).find('[') > 0 else x)

# add the extra information to a new column
capitals_df['city_extra_info'] =  capitals_df['city'].apply(lambda x: x[len(re.search('^(.+?)\(',str(x)).group(1))+1:-1] if str(x).find('(') > 0 else '')

# remove special characters from the city and country names so we can join two datasets later
capitals_df['city'] = capitals_df['city'].apply(lambda x:re.sub('[\',.]','',x).strip())
capitals_df['country'] = capitals_df['country'].apply(lambda x:re.sub('[\',.]','',x).strip())

# convert the city and country to a standard unicode format
capitals_df['city'] = capitals_df['city'].apply(lambda x:unidecode.unidecode(x).strip())
capitals_df['country'] = capitals_df['country'].apply(lambda x:unidecode.unidecode(x).strip())



In [5]:
cities_df.rename(columns={'City Name/s':'city'},inplace=True)
cities_df.rename(columns={'Country/Territory':'country'},inplace=True)
cities_df.rename(columns={'Continental Region':'continent'},inplace=True)
cities_df.rename(columns={'Population':'population'},inplace=True)
cities_df.rename(columns={'Latitude':'latitude'},inplace=True)
cities_df.rename(columns={'Longitude':'longitude'},inplace=True)
cities_df.rename(columns={'Elevation (m)':'elevation'},inplace=True)

# extract all the data before the extra information
cities_df['elevation'] = cities_df['elevation'].apply(lambda x: re.search('^(.+?)\[',str(x)).group(1).strip() if str(x).find('[') > 0 else x)
cities_df['country'] = cities_df['country'].apply(lambda x: re.search('^(.+?)\[',str(x)).group(1).strip() if str(x).find('[') > 0 else x)
cities_df['city'] = cities_df['city'].apply(lambda x: re.search('^(.+?)\[',str(x)).group(1).strip() if str(x).find('[') > 0 else x)

# remove special characters from the city and country names so we can join two datasets later
cities_df['city'] = cities_df['city'].apply(lambda x:re.sub('[\',.]','',x).strip())
cities_df['country'] = cities_df['country'].apply(lambda x:re.sub('[\',.]','',x).strip())

# convert the city and country to a standard unicode format
cities_df['city'] = cities_df['city'].apply(lambda x:unidecode.unidecode(x).strip())
cities_df['country'] = cities_df['country'].apply(lambda x:unidecode.unidecode(x).strip())

In [6]:
cities_df

,country,city,continent,latitude,longitude,population,elevation
0,Nepal,Pokhara,Asia,N28.2096,E83.9856,523000.0,822
1,South Africa,Bloemfontein,Africa,S29.116667,E026.216667,747431.0,1395
2,China,Shanghai,Asia,N31.2304,E121.4737,26320000.0,122
3,Italy,Milan,Europe,N45.4625,E9.186389,1378689.0,122
4,Kazakhstan,Pavlodar,Asia,N52.3000,E76.950000,353930.0,123
...,...,...,...,...,...,...,...
189,Australia,Brisbane,Oceania,S27.4710,E153.0242,NaN,28
190,Australia,Adelaide,Oceania,S34.9286,E138.5986,NaN,61
191,New Zealand,Wellington,Oceania,S41.2865,E174.7762,NaN,20
192,Papua New Guinea,Port Moresby,Oceania,S09.4656,E147.1969,NaN,39


##### Join the two datasets

In [7]:
# join so we can use geo pandas
capitals_with_coordinates = pd.merge(capitals_df,cities_df,how='left',on = ['city','country'])

In [8]:
capitals_with_coordinates

,city,country,notes,city_extra_info,continent,latitude,longitude,population,elevation
0,Abidjan,Ivory Coast (Cote dIvoire),Abidjan is the largest city in Ivory Coast and...,,NaN,NaN,NaN,NaN,NaN
1,Yamoussoukro,Ivory Coast (Cote dIvoire),Abidjan is the largest city in Ivory Coast and...,,NaN,NaN,NaN,NaN,NaN
2,Abu Dhabi,United Arab Emirates,NaN,,Asia,N24.4764,E054.3705,NaN,13
3,Abuja,Nigeria,Lagos was the capital from 1914 to 1991.,,Africa,N09.0580,E007.4891,NaN,777
4,Accra,Ghana,NaN,,Africa,N05.5401,W000.2074,NaN,98
...,...,...,...,...,...,...,...,...,...
255,Windhoek,Namibia,NaN,,Africa,S22.5749,E017.0805,325858.0,1655
256,Yaounde,Cameroon,NaN,,Africa,N03.8612,E011.5217,NaN,726
257,Yaren,Nauru,"Nauru has no official capital; however, the go...",,NaN,NaN,NaN,NaN,NaN
258,Yerevan,Armenia,NaN,,Asia,N40.1596,E044.5090,NaN,1032


##### Quality Assurance

In [9]:
#TODO Clean up the the city and country more so we can better join, include city/country with accents column
issues = capitals_with_coordinates[(capitals_with_coordinates['population'].isna()) & (capitals_with_coordinates['continent'].isna())]
issues.to_csv('issues.csv',encoding='utf-16',index=False)

In [10]:
capitals_with_coordinates.to_csv('capitals.csv',header=True,index=False,encoding='utf-16')
capitals_csv = pd.read_csv('capitals.csv',encoding='utf-16')

##### Create a SQL Lite database

In [11]:
conn = sqlite3.connect('capitals.db')
cc_cursor = sqlite3.Cursor(conn)


In [12]:
drop_table = 'DROP TABLE IF EXISTS capitals'

In [13]:
cc_cursor.execute(drop_table)

In [14]:
create_table = '''CREATE TABLE capitals (
                    city VARCHAR(250),
                    city_extra_info VARCHAR(250),
                    country VARCHAR(250),
                    notes VARCHAR(500),
                    continent VARCHAR(20),
                    latitude VARCHAR(30),
                    longitude VARCHAR(30),
                    population NUMERIC(15),
                    elevation NUMERIC(10)
            
)'''

In [15]:
cc_cursor.execute(create_table)

In [16]:
capitals_csv.to_sql('capitals',conn,if_exists='append',index=False)

260

In [17]:
view_snapshot = pd.read_sql('SELECT * FROM capitals LIMIT 20',conn)

In [18]:
view_snapshot

,city,city_extra_info,country,notes,continent,latitude,longitude,population,elevation
0,Abidjan,None,Ivory Coast (Cote dIvoire),Abidjan is the largest city in Ivory Coast and...,None,None,None,NaN,NaN
1,Yamoussoukro,None,Ivory Coast (Cote dIvoire),Abidjan is the largest city in Ivory Coast and...,None,None,None,NaN,NaN
2,Abu Dhabi,None,United Arab Emirates,None,Asia,N24.4764,E054.3705,NaN,13.0
3,Abuja,None,Nigeria,Lagos was the capital from 1914 to 1991.,Africa,N09.0580,E007.4891,NaN,777.0
4,Accra,None,Ghana,None,Africa,N05.5401,W000.2074,NaN,98.0
5,Adamstown,None,Pitcairn Islands,British Overseas Territory.,None,None,None,NaN,NaN
6,Addis Ababa,None,Ethiopia,None,Africa,N09.0084,E038.7575,3384569.0,2362.0
7,Aden,None,Yemen,"Due to the Yemeni civil war (2014–present), Sa...",None,None,None,NaN,NaN
8,Sanaa,None,Yemen,"Due to the Yemeni civil war (2014–present), Sa...",Asia,N15.3556,E044.2081,1937451.0,2253.0
9,Algiers,None,Algeria,None,None,None,None,NaN,NaN


##### Countries with more than 2 capitals

In [19]:
more_than_2_capitals = pd.read_sql( '''SELECT country,city FROM capitals
                                    WHERE country IN (SELECT country
                                                    FROM capitals 
                                                    GROUP BY country
                                                    HAVING count(*) > 1)
                          ''',conn)

In [20]:
more_than_2_capitals

,country,city
0,Ivory Coast (Cote dIvoire),Abidjan
1,Ivory Coast (Cote dIvoire),Yamoussoukro
2,Yemen,Aden
3,Yemen,Sanaa
4,Netherlands,Amsterdam
5,Netherlands,The Hague
6,South Africa,Bloemfontein
7,South Africa,Cape Town
8,South Africa,Pretoria
9,Montserrat,Brades


##### Top 5 cities by elevation

In [21]:
top_5_elevation = pd.read_sql('''SELECT * FROM 
                              (SELECT *,RANK() OVER(ORDER BY elevation DESC) AS ranker
                              FROM capitals
                              )
                              WHERE ranker < 6

''',conn)

In [22]:
top_5_elevation

,city,city_extra_info,country,notes,continent,latitude,longitude,population,elevation,ranker
0,La Paz,None,Bolivia,La Paz is the highest administrative capital (...,Americas,-16.500,-68.150,877363,3812,1
1,Quito,None,Ecuador,"Highest official capital (2,850 m).[5]",Americas,S00.2295,W078.5243,2671191,2850,2
2,Bogota,None,Colombia,None,Americas,N04.6473,W074.0962,7878783,2619,3
3,Asmara,None,Eritrea,None,Africa,N15.3315,E038.9183,649000,2363,4
4,Addis Ababa,None,Ethiopia,None,Africa,N09.0084,E038.7575,3384569,2362,5


##### Most populous capital in each continent

In [23]:
most_populous_continent = pd.read_sql('''SELECT city,country,continent,population FROM 
                              (SELECT *,RANK() OVER(PARTITION BY continent ORDER BY population DESC) AS ranker
                              FROM capitals
                              WHERE population IS NOT NULL AND population IS NOT NULL
                              )
                              WHERE ranker = 1

''',conn)

In [24]:
most_populous_continent

,city,country,continent,population
0,Addis Ababa,Ethiopia,Africa,3384569
1,Mexico City,Mexico,Americas,8918653
2,Manila,Philippines,Asia,12877253
3,Minsk,Belarus,Europe,1982444


In [25]:
# conn.close()